In [1]:
import numpy as np
from utils.dataloader import *
from utils.coordinate_calibration import PointCloudRegistration
from utils.meshgrid import Meshgrid
from utils.icp import IterativeClosestPoint, Matching
import os
from test import FileOutputMatcher

In [2]:
OUTPUT_DIR = './OUTPUT'
DATA_DIR = './pa345_data'

RIGID_BODY_DATA_A = f'{DATA_DIR}/Problem3-BodyA.txt'
RIGID_BODY_DATA_B = f'{DATA_DIR}/Problem3-BodyB.txt'

SURFACE_DATA = f'{DATA_DIR}/Problem3MeshFile.sur'

SAMPLE_ID = 'B'
SAMPLE_DATA = f'./pa345_data/PA3-{SAMPLE_ID}-Debug-SampleReadingsTest.txt'
# SAMPLE_DATA = f'./pa345_data/PA3-{SAMPLE_ID}-Unknown-SampleReadingsTest.txt'

# Load data files
rigidbody_dl_A = RigidBodyDataloader.read_file(RIGID_BODY_DATA_A)
rigidbody_dl_B = RigidBodyDataloader.read_file(RIGID_BODY_DATA_B)

surface_dl = Surfaceloader.read_file(SURFACE_DATA)
sample_dl = SampleReadingsDataloader.read_file(SAMPLE_DATA, delimiter=',', N_A=rigidbody_dl_A.N_markers, N_B=rigidbody_dl_B.N_markers)

### Get F<sub>A, k</sub> and F<sub>B, k</sub>

In [3]:
rigidbody_dl_A_markers = rigidbody_dl_A.markers.reshape(1, -1, 3) # markers of body A in body A coordinates

sample_dl_A = sample_dl.body_A # samples of body A markers in tracker coordinates
num_samples = sample_dl.N_samps

# perform registration for each frame
reg = PointCloudRegistration()
F_A = []
for i in range(num_samples):
    sample_dl_A_i = sample_dl_A[i].reshape(1, -1, 3)
    F_A_i, err = reg.register(rigidbody_dl_A_markers, sample_dl_A_i)
    F_A.append(F_A_i)

F_A = np.array(F_A)

ridigbody_dl_B_markers = rigidbody_dl_B.markers.reshape(1, -1, 3) # markers of body B in body B coordinates
sample_dl_B = sample_dl.body_B # samples of body B markers in tracker coordinates

# perform registration for each frame
F_B = []
for i in range(num_samples):
    sample_dl_B_i = sample_dl_B[i].reshape(1, -1, 3)

    F_B_i, err = reg.register(ridigbody_dl_B_markers, sample_dl_B_i)
    F_B.append(F_B_i)

F_B = np.array(F_B)

Performing 3D point cloud registration...


100ep [00:01<00:00, 59.93it/s, err=0.00281]


Done.
Performing 3D point cloud registration...


100ep [00:00<00:00, 131.28it/s, err=0.00266]


Done.
Performing 3D point cloud registration...


...

Done.
Performing 3D point cloud registration...


100ep [00:00<00:00, 167.51it/s, err=0.00226]


Done.
Performing 3D point cloud registration...


100ep [00:00<00:00, 358.38it/s, err=0.00226]

Done.


### Get d<sub>k

In [4]:
A_tip = rigidbody_dl_A.tip
A_tip = np.append(A_tip, 1) # add 1 for homogenous coordinates
d_k = []

for i in range(num_samples):
    F_A_i = F_A[i] # get F_A for frame i
    F_B_i_inv = np.linalg.inv(F_B[i]) # get F_B inverse for frame i

    d_k.append(F_B_i_inv @ F_A_i @ A_tip) # d_k = F_B^-1 * F_A * A_tip

d_k = np.array(d_k)[:,:3]
print(d_k.shape)
print(d_k)

(15, 3)
[[-37.64833678  -9.48581429 -12.76682898]
 [-18.60510135  -9.65676445  -4.29505254]
 [ -2.14008856  -7.91886808  52.44877509]
 [ 17.87241499 -12.57782081   1.66919045]
 [-13.95441346  21.50282627  25.2969365 ]
 [ 28.96426532  19.0550331  -13.73958009]
 [ 24.2614743   22.12267255  -1.99781554]
 [-27.75457871 -23.80052244 -48.36744324]
 [-28.46403537   7.90253067 -35.89238329]
 [-12.65584902   9.56599089  12.90058673]
 [-27.11807733 -28.65576806 -14.35566804]
 [ 37.25503378   5.25779861   4.71164992]
 [ 20.17629056 -14.87243941  -4.87098927]
 [ -4.2582231   12.83878122  54.08074924]
 [ 32.08328176  15.8541175   -2.09464545]]


In [ ]:
# Select algorithm for finding closest point
# Choices: SIMPLE_LINEAR, VECTORIZED_LINEAR, SIMPLE_OCTREE, VECTORIZED_OCTREE
matching_algo = Matching.VECTORIZED_LINEAR  # VECTORIZED_LINEAR is the fastest

# Initialize ICP helper class 
icp = IterativeClosestPoint(matching_algo)

# Initialize meshgrid of Triangles
mesh = Meshgrid(surface_dl.vertices, surface_dl.triangles)

# Find closest points and euclidean distances
pt, dist = icp.match(d_k, mesh)

print(f'Closest points: {np.around(pt, 2)}')
print(f'Closest distances: {np.around(dist, 3)}')

Closest points: [[-38.82  -9.64 -11.91]
 [-19.12  -9.7   -2.94]
 [ -1.19  -5.73  52.65]
 [ 17.86 -12.56   1.66]
 [-14.6   22.22  25.28]
 [ 30.81  20.12 -13.97]
 [ 25.46  24.26  -2.07]
 [-27.05 -21.42 -45.7 ]
 [-28.55   8.08 -36.  ]
 [-11.2    8.79  13.3 ]
 [-27.08 -28.51 -14.42]
 [ 39.65   4.99   4.85]
 [ 20.58 -15.33  -4.37]
 [ -4.79  13.09  54.15]
 [ 33.31  16.96  -2.28]]
Closest distances: [1.463 1.449 2.399 0.021 0.969 2.14  2.455 3.641 0.226 1.698 0.164 2.412
 0.79  0.588 1.657]


### write output to file

In [6]:
output_file = f'{OUTPUT_DIR}/pa3-{SAMPLE_ID}-Output.txt'
with open(output_file, 'w') as file:
    file.write(f"{num_samples}, {output_file}\n")
    for sample in range(num_samples):
        file.write(f"{d_k[sample][0]:.2f} {d_k[sample][1]:.2f} {d_k[sample][2]:.2f} ")
        file.write(f"{pt[sample][0]:.2f} {pt[sample][1]:.2f} {pt[sample][2]:.2f}")
        file.write(f" {dist[sample]:.2f}")
        file.write("\n")

print(f"Output written to {output_file}")


Output written to ./OUTPUT/pa3-B-Output.txt


In [7]:
for SAMPLE_ID in ['A', 'B', 'C', 'D', 'E', 'F']:
    pred_file = f'{OUTPUT_DIR}/pa3-{SAMPLE_ID}-Output.txt'
    gt_file = f'{DATA_DIR}/PA3-{SAMPLE_ID}-Debug-Output.txt'

    if os.path.exists(pred_file) and os.path.exists(gt_file):
        matcher = FileOutputMatcher()

        error_d_k, error_c_k, error_norm = matcher(pred_file, gt_file)

        print(f'Error analysis for sample {SAMPLE_ID}:')
        print(f'MAE of pointer tip in body B coordinates: {error_d_k}')
        print(f'MAE of closest point on surface in CT coordinates: {error_c_k}')
        print(f'MAE of the norm of the difference: {error_norm}')
        print("\n")
    else:
        print('No prediction or ground-truth file found. Skipping operation.')

Error analysis for sample A:
MAE of pointer tip in body B coordinates: (0.0038451622, 0.0072913724, 0.0018814723)
MAE of closest point on surface in CT coordinates: (0.0031657636, 0.005367279, 0.0013563792)
MAE of the norm of the difference: 0.0032665999606251717


Error analysis for sample B:
MAE of pointer tip in body B coordinates: (0.0013332367, 0.00533336, 0.0006666819)
MAE of closest point on surface in CT coordinates: (0.0019999743, 0.0013333638, 0.0)
MAE of the norm of the difference: 0.0032666500192135572


Error analysis for sample C:
MAE of pointer tip in body B coordinates: (0.002441295, 0.004419346, 0.003953846)
MAE of closest point on surface in CT coordinates: (0.0033328296, 0.0038325726, 0.002295367)
MAE of the norm of the difference: 0.0020036071073263884


Error analysis for sample D:
MAE of pointer tip in body B coordinates: (0.0046200277, 0.0049936534, 0.0020193835)
MAE of closest point on surface in CT coordinates: (0.0032187144, 0.0030613265, 0.0026649297)
MAE of 